## Import Dependencies

In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy
import Tasmanian

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle
import gzip

warnings.simplefilter("ignore", category=RuntimeWarning)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.54 s, sys: 349 ms, total: 2.89 s
Wall time: 2.96 s


## Define Method and DateTime

In [2]:
#Generate Method (except last part)
meth_name = Method_name_enum(5)
method = GPBO_Methods(meth_name)

print("Method Name: ", method.method_name.name)
print("Emulator?: ", method.emulator)
print("Obj Func: ", method.obj.name)
print("Sparse Grid?: ", method.sparse_grid)

#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

Method Name:  C2
Emulator?:  True
Obj Func:  OBJ
Sparse Grid?:  True


# Case Study GPBO Driver Test

In [3]:
cs_name_val = 1
CS_name  = CS_name_enum(cs_name_val)
print("CS Name: ", CS_name.name)

param_name_str = set_param_str(cs_name_val)
print(param_name_str)
indecies_to_consider = set_idcs_to_consider(cs_name_val, param_name_str)
print(len(indecies_to_consider))

ep0 = 1
ep_enum = Ep_enum(1)
sep_fact = 1.0
normalize = False
noise_mean = 0
noise_std = 0.01
# noise_std = 0.0
kernel = Kernel_enum(1)
lenscl = None
outputscl = 1
retrain_GP = 1
reoptimize_obj = 5
bo_iter_tot = 15
bo_run_tot = 1
save_data = False
seed = 1
ei_tol = 1e-6
obj_tol = 1e-6

num_x_data = 5
gen_meth_x = Gen_meth_enum(2) #Note: Has to be the same for validation and sim data
num_theta_data = 10*len(indecies_to_consider)
gen_meth_theta = Gen_meth_enum(1)

CS Name:  CS1
t1t2
2


In [4]:
simulator = simulator_helper_test_fxns(CS_name, indecies_to_consider, noise_mean, noise_std, normalize, seed)

#Generate Exp Data
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)

#Generate Sim Data
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, False)
sim_sse_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, sep_fact, False)

#Generate Validation Data (for small case studies only)
if cs_name_val <= 2:
    gen_heat_map_data = True
    num_theta_data_val = 20
    gen_meth_theta_val = Gen_meth_enum(2)
    val_data = simulator.gen_sim_data(num_theta_data_val, num_x_data, gen_meth_theta_val, gen_meth_x, sep_fact, True)
    val_sse_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, sep_fact, True)
    
else:
    num_theta_data_val = 0
    gen_meth_theta_val = None
    val_data = None
    val_sse_data = None
    gen_heat_map_data = False

#Set Ep_Bias
if ep_enum.value == 1:
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, None, None, None, None, None)
elif ep_enum.value == 2:
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, bo_iter_tot, None, 0.5, None, None, None)
elif ep_enum.value == 3:
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, 1.5, None, None, None, None)
else:
    ep_bias = Exploration_Bias(None, None, ep_enum, None, None, None, None, None, None, None)
    
#Set Cs_params and Simulator
cs_name = CS_name.name + "_BO_method_" + meth_name.name + "_sep_fact_" + str(round(sep_fact,2))
cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, kernel, lenscl, outputscl, retrain_GP, 
                                reoptimize_obj, gen_heat_map_data, bo_iter_tot, bo_run_tot, save_data, DateTime, 
                                seed, ei_tol, obj_tol)

#Initialize Driver
driver = GPBO_Driver(cs_params, method, simulator, exp_data, sim_data, sim_sse_data, val_data, val_sse_data, None, 
                     ep_bias, gen_meth_theta)

In [5]:
#Run Bo iterations/restarts
restart_bo_results = driver.run_bo_restarts()

In [6]:
#Define the run number and bo iter in question and pull the max bo iters and runs
run_num = -1
bo_iter = -1
runs = restart_bo_results[run_num].configuration["Number of Workflow Restarts"]
num_sets = restart_bo_results[run_num].configuration["Max BO Iters"]

#Print Results
print("")
print(restart_bo_results[run_num].configuration)
# print(loaded_results[run_num].list_gp_emulator_class[bo_iter])
# print(len(loaded_results[run_num].list_heat_map_data))
print(restart_bo_results[run_num].why_term)
print(restart_bo_results[run_num].results_df)
print(restart_bo_results[run_num].max_ei_details_df)
print(restart_bo_results[run_num].results_df["Theta Min Obj Cum."].iloc[-1])
print(restart_bo_results[run_num].simulator_class.theta_true)
print(restart_bo_results[run_num].list_gp_emulator_class[bo_iter].trained_hyperparams)


{'DateTime String': None, 'Method Name Enum Value': 5, 'Case Study Name': 'CS1_BO_method_C2_sep_fact_1.0', 'Number of Parameters': 2, 'Exploration Bias Method Value': 1, 'Separation Factor': 1.0, 'Normalize': False, 'Initial Kernel': <Kernel_enum.MAT_52: 1>, 'Initial Lengthscale': None, 'Initial Outputscale': 1, 'Retrain GP': 1, 'Reoptimize Obj': 5, 'Heat Map Points Generated': True, 'Max BO Iters': 15, 'Number of Workflow Restarts': 1, 'Seed': 1, 'EI Tolerance': 1e-06, 'Obj Improvement Tolerance': 1e-06, 'Theta Generation Enum Value': 1}
ei
   index  Best Error  Exploration Bias     Max EI  \
0      0    5.342819               1.0   0.177168   
1      1    5.342819               1.0   0.000000   
2      2    5.342819               1.0  28.987063   
3      3    0.001722               1.0   0.000000   
4      4    0.001722               1.0   0.000000   

                                Theta Max EI   Min Obj  Min Obj Act  \
0  [0.9693773371266958, -1.5716532808912487]  0.000312     0.